In [65]:
#Importing libraries needed for the project
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score

In [4]:
data=pd.read_csv("/home/kasia/data_ML/ML_data.csv")

In [16]:
data.set_index("id")

data["diagnosis_binary"]=data["diagnosis"]
data["diagnosis_binary"]=data["diagnosis_binary"].replace("B",0)
data["diagnosis_binary"]=data["diagnosis_binary"].replace("M",1)
data_ML=data.drop("diagnosis", axis=1)

/tmp/ipykernel_5452/2402691581.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["diagnosis_binary"]=data["diagnosis_binary"].replace("M",1)


In [28]:
X=data_ML.iloc[:,1:31]
Y=data_ML["diagnosis_binary"]

In [37]:
Y.value_counts()
#there is a class imbalance in the data - more classified as 0 than 1
#to correct for class imbalance SMOTE is used
smote=SMOTE(random_state=42, sampling_strategy="minority")
X_res, Y_res=smote.fit_resample(X,Y)

In [58]:
#Splitting the data into the training and testing datasets, by default test is 25% of the data
X_train, X_test, Y_train, Y_test = train_test_split(X_res, Y_res)

In [70]:
#Comparing ML classifiers for their potential to predict the class based on the features
scores={"Accuracy":[], "F1":[], "Recall":[]}

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(),
    SVC(kernel="linear", random_state=42),
    SVC(kernel="rbf", random_state=42),
    GaussianProcessClassifier(random_state=42),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier( random_state=42),
    MLPClassifier(random_state=42),
    AdaBoostClassifier(random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

for classifier in classifiers:
    pipe=make_pipeline(StandardScaler(), classifier)
    pipe.fit(X_train, Y_train)
    prediction=pipe.predict(X_test)
    accuracy_score(prediction, Y_test)
    f1_score(prediction, Y_test)
    recall_score(prediction, Y_test)

/home/kasia/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kasia/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [46]:
enumerate(classifiers)